In [ ]:
import transformers

#Set to avoid warning messages.
transformers.logging.set_verbosity_error()


/opt/anaconda3/envs/transformers-2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 04.02. Loading a Hugging Face Dataset

In [ ]:
from datasets import load_dataset

#Use pretrained model checkpoint from Huggingface
model_name = "distilbert-base-uncased"
#Use pre-labeled dataset from huggingface
dataset_name= "poem_sentiment"

poem_sentiments = load_dataset(dataset_name)

#Apache Arrow format
print(poem_sentiments)
print(poem_sentiments["test"][20:25])

print("\nSentiment Labels used",
      poem_sentiments["train"].features.get("label").names)

Found cached dataset poem_sentiment (/Users/linkedin/.cache/huggingface/datasets/poem_sentiment/default/1.0.0/4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099)
100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 438.72it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 892
    })
    validation: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 105
    })
    test: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 104
    })
})
{'id': [20, 21, 22, 23, 24], 'verse_text': ["as o'er the earth it wanders wide,", 'how hearts were answering to his own,', 'glad on its stone-built hearth; and thorough the wide-mouthed smoke-flue', 'sees the clouds reel and roll above our head,', '’tis to behold his vengeance for my son.'], 'label': [2, 1, 2, 2, 0]}

Sentiment Labels used ['negative', 'positive', 'no_impact', 'mixed']


## 04.03. Encoding and pre-processing the dataset

In [ ]:
#Encoding text

from transformers import DistilBertTokenizer

db_tokenizer = DistilBertTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return db_tokenizer(batch["verse_text"],
                        padding=True,
                        truncation=True)

enc_poem_sentiment = poem_sentiments.map(
                        tokenize,
                        batched=True,
                        batch_size=None)

print(enc_poem_sentiment["train"][0:5])


Loading cached processed dataset at /Users/linkedin/.cache/huggingface/datasets/poem_sentiment/default/1.0.0/4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099/cache-e80e4ec2cbfe3b03.arrow
Loading cached processed dataset at /Users/linkedin/.cache/huggingface/datasets/poem_sentiment/default/1.0.0/4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099/cache-781d7903f5c80b58.arrow
Loading cached processed dataset at /Users/linkedin/.cache/huggingface/datasets/poem_sentiment/default/1.0.0/4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099/cache-826927031abc85e4.arrow


{'id': [0, 1, 2, 3, 4], 'verse_text': ['with pale blue berries. in these peaceful shades--', 'it flows so long as falls the rain,', 'and that is why, the lonesome day,', 'when i peruse the conquered fame of heroes, and the victories of mighty generals, i do not envy the generals,', 'of inward strife for truth and liberty.'], 'label': [1, 2, 0, 3, 3], 'input_ids': [[101, 2007, 5122, 2630, 22681, 1012, 1999, 2122, 9379, 13178, 1011, 1011, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2009, 6223, 2061, 2146, 2004, 4212, 1996, 4542, 1010, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1998, 2008, 2003, 2339, 1010, 1996, 10459, 14045, 2154, 1010, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2043, 1045, 7304, 3366, 1996, 11438, 4476, 1997, 7348, 1010, 1998, 1996, 9248, 1997, 10478, 11593, 1010, 1045, 2079, 2025, 21103, 1996, 11593, 1010, 102, 0, 0], [101, 1997, 20546, 27865, 2005, 3606, 1998, 7044, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
#Explore input IDs and Attention Mask

print("Text :",
      enc_poem_sentiment["train"][1].get("verse_text"))
print("\nInput Map :",
      enc_poem_sentiment["train"][1].get("input_ids"))
print("\nAttention Mask :",
      enc_poem_sentiment["train"][1].get("attention_mask"))

print("\nTotal tokens: ",
      len(enc_poem_sentiment["train"][1].get("input_ids")))
print("Non Zero tokens: ",
      len(list(filter(
        lambda x :x > 0,
          enc_poem_sentiment["train"][1].get("input_ids")))))
print("Attention = 1: ",
      len(list(filter(
        lambda x :x > 0,
          enc_poem_sentiment["train"][1].get("attention_mask")))))

Text : it flows so long as falls the rain,

Input Map : [101, 2009, 6223, 2061, 2146, 2004, 4212, 1996, 4542, 1010, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Attention Mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Total tokens:  28
Non Zero tokens:  11
Attention = 1:  11


In [ ]:
#Separate training and validation sets
training_dataset = enc_poem_sentiment["train"]
validation_dataset=enc_poem_sentiment["validation"]

print("\nColumn Names : ",training_dataset.column_names)
print("\nFeatures : ",training_dataset.features)

labels = training_dataset.features.get("label")
num_labels=len(labels.names)



Column Names :  ['id', 'verse_text', 'label', 'input_ids', 'attention_mask']

Features :  {'id': Value(dtype='int32', id=None), 'verse_text': Value(dtype='string', id=None), 'label': ClassLabel(names=['negative', 'positive', 'no_impact', 'mixed'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}
